In [67]:
from datetime import datetime, date, timedelta
import pandas as pd
import numpy as np
import math
import os
print(os.getcwd())


c:\Users\Joshualevy\Documents\capitalisntDashboard


In [ ]:
#### 
def most_recent_collection(year: str, month: str, date: str) -> pd.DataFrame:
    """
    
    """
    cities_path_base = 'us_cities_episode_locations-{yyyy}-{mm}-{dd}.csv'.format(
        yyyy = year,
        mm = month,
        dd = date
    )

    df = pd.read_csv(cities_path_base)
    return df




In [89]:
def generate_interp_dates(start_date: str, end_date: str) -> list:
    """
    qwer
    """

    start_date_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_date_dt = datetime.strptime(end_date, '%Y-%m-%d')

    delta = end_date_dt - start_date_dt

    # print(start_date_dt)
    # print(end_date_dt)
    # print(delta.days)
    if delta.days <= 1:
        raise Exception("Hey, I think your dates don't require interpolation!")

    interp_dates_list = []
    for i in range(1, delta.days):
        new_day = start_date_dt + timedelta(days=i)
        new_day_str = datetime.strftime(new_day, '%Y-%m-%d')
        interp_dates_list.append(new_day_str)
        


    return interp_dates_list


['2022-09-08', '2022-09-09', '2022-09-10']

In [140]:
def construct_interpolate_df(df: pd.DataFrame, start_interp_date: str, stop_interp_date) -> pd.DataFrame:
    """
    asdf
    """
    
    dates_to_interp_list = generate_interp_dates(start_interp_date, stop_interp_date)


    out_df = df.copy()
    out_df = out_df.set_index(['city_name', 'city_id', 'state_id', 'episode_id'])
    out_df[dates_to_interp_list] = np.nan

    date_cols = list(out_df.columns)

    for index, date_str in enumerate(date_cols):
        if date_str == start_interp_date:
            split_open_index = index + 1
            continue
        elif date_str == stop_interp_date:
            split_close_index = index
            break

    pre_interp_dates_list = date_cols[0 : split_open_index]
    post_interp_dates_list = date_cols[split_close_index : -len(dates_to_interp_list)]

    ordered_interp_date_cols = pre_interp_dates_list + dates_to_interp_list + post_interp_dates_list
    out_df = out_df[ordered_interp_date_cols]

    out_df = out_df.interpolate(
        method = 'linear',
        axis = 'columns',
        limit_direction = 'forward',
        limit_area = 'inside'
    )

    return out_df



In [150]:
def interp_df_rebiasing(df: pd.DataFrame, start_interp_date: str, stop_interp_date: str) -> pd.DataFrame:
    """
    asdf
    """

    dates_to_rebias_list = generate_interp_dates(start_interp_date, stop_interp_date)

    if len(dates_to_rebias_list) == 1:
        interest_date = dates_to_rebias_list[0]
        df[interest_date] = df[[start_interp_date, interest_date, stop_interp_date]].apply(
            lambda obs: linear_interp_biasing(obs)
        )
    else:
        for index, interest_date in enumerate(dates_to_rebias_list):
            if index == 0:
                df[interest_date] = df[[start_interp_date, interest_date, dates_to_rebias_list[index + 1]]].apply(
                    lambda obs: linear_interp_biasing(obs),
                    axis = 'columns'
                )
            elif index == len(dates_to_rebias_list) - 1:
                df[interest_date] = df[[dates_to_rebias_list[index - 1], interest_date, stop_interp_date]].apply(
                    lambda obs: linear_interp_biasing(obs),
                    axis = 'columns'
                )
            else:
                df[interest_date] = df[[dates_to_rebias_list[index - 1], interest_date, dates_to_rebias_list[index + 1]]].apply(
                    lambda obs: linear_interp_biasing(obs),
                    axis = 'columns'
                )

    return df


    

In [141]:
def linear_interp_biasing(observation : pd.Series) -> float:
    """
    """

    pre = observation.iloc[0]
    interest = observation.iloc[1]
    post = observation.iloc[2]

    if pre == interest:
        return interest
    
    if pre > interest:
        return pre
    elif interest.is_integer():
        return interest


    if pre < interest:
        if not interest.is_integer():
            # print((pre, interest, post))
            # print('NEW VALUE SHOULD BE: {}'.format(math.ceil(post)))
            return math.ceil(post)
            
    return interest



In [160]:
year = '2022'
month = '09'
date = '12'

missing_df = most_recent_collection(year, month, date)


new_df = missing_df.copy()
new_df = construct_interpolate_df(new_df, '2022-09-07', '2022-09-11')
new_df = interp_df_rebiasing(new_df, '2022-09-07', '2022-09-11')
new_df = new_df.reset_index(drop=False)
new_df.to_csv('us_cities_episode_locations-{}-{}-{}-INTERPOLATED.csv'.format(
    year, month, date),
    index=False,
    encoding='utf-8'
)
new_df

Dates to rebias after linear-interpolation
['2022-09-08', '2022-09-09', '2022-09-10']
0 2022-09-08
1 2022-09-09
2 2022-09-10


,city_name,city_id,state_id,episode_id,2022-08-09,2022-08-10,2022-08-11,2022-08-12,2022-08-13,2022-08-14,...,2022-09-03,2022-09-04,2022-09-05,2022-09-06,2022-09-07,2022-09-08,2022-09-09,2022-09-10,2022-09-11,2022-09-12
0,NaN,11961466.0,6254928,7215fbfa-8fde-4893-b0d5-94c88b1bd831,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,NaN,11961466.0,6254928,bc91e1ee-86e4-4137-800a-db4b6906e6c6,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,NaN,11961466.0,6254928,e7be761a-a4ee-4b6d-95d5-8df72c6aee38,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,'Aiea,5856430.0,5855797,0c02cbae-88aa-4f4a-a3e6-3ad0ce8971f6,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,'Aiea,5856430.0,5855797,15f4339a-889f-4dfc-8ca8-c26ee2788e35,2.0,2.0,2.0,2.0,2.0,2.0,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194900,West Dundee,4915977.0,4896861,e87b9658-6aeb-4837-9e96-a0b41e3c90e9,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
194901,West Dundee,4915977.0,4896861,ea26a086-b538-4a95-81c8-fce31abc4708,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
194902,West Dundee,4915977.0,4896861,f265af72-e965-4988-9a4f-29259533859f,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
194903,West Dundee,4915977.0,4896861,f44adca5-f06c-483d-bacc-e786eca46907,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
